# install transformers

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 74.1 MB/s 
     |████████████████████████████████| 7.6 MB 66.3 MB/s 


# example

In [7]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ainize/klue-bert-base-mrc")
model = AutoModelForQuestionAnswering.from_pretrained("ainize/klue-bert-base-mrc")

context = "명탐정 코난은 1996년에 방영이 시작된 애니메이션으로 고등학생 명탐정 쿠도 신이치가 검은 조직의 거래현장을 목격하다가 습격당하여 어린아이로 변한 후의 이야기이다. 전형적인 탐정물이며, 에도가와 코난이란 이름으로 활동하게 되는데, 이런 명대사가 있다. '에도가와 코난, 탐정이지.' 코난이 등장하는 곳은 살인사건이 일어나는데 시청자로써는 끝나지 않는 애니메이션이라 좋은 점도 있지만 많은 사람들이 죽어나가는 것을 보면 조금 안타깝기도 하다."
question = "요약하면?"

encodings = tokenizer(context, question, max_length=512, truncation=True,
                      padding="max_length", return_token_type_ids=False)
encodings = {key: torch.tensor([val]) for key, val in encodings.items()}             

input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]

pred = model(input_ids, attention_mask=attention_mask)

start_logits, end_logits = pred.start_logits, pred.end_logits

token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)

pred_ids = input_ids[0][token_start_index: token_end_index + 1]

prediction = tokenizer.decode(pred_ids)
prediction

"에도가와 코난, 탐정이지.'코난이 등장하는 곳은 살인사건"

In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.1 MB/s 
     |████████████████████████████████| 213 kB 65.1 MB/s 
     |████████████████████████████████| 132 kB 68.0 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = "명탐정 코난은 1996년에 방영이 시작된 애니메이션으로 고등학생 명탐정 쿠도 신이치가 검은 조직의 거래현장을 목격하다가 습격당하여 어린아이로 변한 후의 이야기이다. 전형적인 탐정물이며, 에도가와 코난이란 이름으로 활동하게 되는데, 이런 명대사가 있다. '에도가와 코난, 탐정이지.' 코난이 등장하는 곳은 살인사건이 일어나는데 시청자로써는 끝나지 않는 애니메이션이라 좋은 점도 있지만 많은 사람들이 죽어나가는 것을 보면 조금 안타깝기도 하다."

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

에도가와 코난이 등장하는 탐정물.


# load data

In [3]:
train_file_path = '/content/drive/MyDrive/Alpaco_NLP/training/tl_news.pkl'
test_file_path = '/content/drive/MyDrive/Alpaco_NLP/validation/vl_news.pkl'

import pickle

with open(train_file_path, 'rb') as f:
    df_train = pickle.load(f)

with open(test_file_path, 'rb') as f:
    df_test = pickle.load(f)

In [4]:
X_train_list = df_train['original'].tolist()
X_test_list = df_test['original'].tolist()
y_train = df_train['summary'].tolist()
y_test = df_test['summary'].tolist()

In [16]:
# save data to tsv file
import csv
from tqdm import tqdm

with open('/content/drive/MyDrive/Alpaco_NLP/train.tsv', 'w', encoding='utf-8', newline='') as f:
    tw = csv.DictWriter(f, fieldnames = ["news", "summary"], delimiter='\t')
    tw.writeheader()
    for original, summary in zip(tqdm(df_train['original']), df_train['summary']):
        tw.writerow({'news': original.replace('\n', ' '), 'summary': summary.replace('\n', ' ')})
with open('/content/drive/MyDrive/Alpaco_NLP/test.tsv', 'w', encoding='utf-8', newline='') as f:
    tw = csv.DictWriter(f, fieldnames = ["news", "summary"], delimiter='\t')
    tw.writeheader()
    for original, summary in zip(tqdm(df_test['original']), df_test['summary']):
        tw.writerow({'news': original.replace('\n', ' '), 'summary': summary.replace('\n', ' ')})



100%|██████████| 21600/21600 [00:01<00:00, 12234.18it/s]

100%|██████████| 2700/2700 [00:00<00:00, 13068.78it/s]


In [4]:
import pandas as pd
df_train = pd.read_csv("/content/drive/MyDrive/Alpaco_NLP/train.tsv", delimiter='\t')
df_test = pd.read_csv("/content/drive/MyDrive/Alpaco_NLP/test.tsv", delimiter='\t')

# load model, tokenizer

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TFAutoModelForSeq2SeqLM, AutoModelForSeq2SeqLM

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [6]:
model.summary()

AttributeError: ignored

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_scheduler
from datasets import load_metric

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

class SummaryDataset(Dataset):
    def __init__(
        self,
        data,
        tokenizer=tokenizer,
        text_max_token_len = 256,
        summary_max_token_len = 128
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['news']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

train_dataset = SummaryDataset(data=df_train)
test_dataset = SummaryDataset(data=df_test)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)
eval_dataloader = DataLoader(test_dataset, batch_size=2)

# train

https://www.kaggle.com/code/ibnmohey/mt5-multilingual-xlsum-cbd321

In [ ]:
num_epochs = 1

num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         print(predictions)
#         print(batch["labels"])
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'/content/drive/MyDrive/Alpaco_NLP/t5-kor.pth')

    print(f'epoch: {epoch + 1} -- loss: {loss}')

In [9]:
import torch
d = torch.load('/content/drive/MyDrive/Alpaco_NLP/t5-kor.pth')
d

Output hidden; open in https://colab.research.google.com to view.

In [10]:
d.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])

In [11]:
model.load_state_dict(d['model_state_dict'])

<All keys matched successfully>

In [12]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [14]:
text = '''
노형욱 장관, 철도역 방역ㆍ운행 안전에 만전강조\n 14일 서울역 KTX 철도역사열차 방역실태 전반에 걸쳐 현장 점검\n 철도특별사법경찰대에는 방역수칙 위반자에 대한 무관용 원칙 대응 주문 \n 노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다.\n 먼저, 노 장관은 한국철도공사로부터 수도권 거리두기 4단계 격상에 따른 전국 주요 역사 탑승 전 발열체크 등 강화된 철도분야 방역대책을 보고 받은 뒤, 코로나19 로 어려운 상황 속에서도 국민을 위해 힘써 주시는 노고에 감사드린다면서 관계자들을 격려하였다.\n 이어, KTX 방역현장을 둘러본 뒤, 지금까지 철도분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다라며, 강화된 방역대책이 승객에게 잘 안내되고, 이를 통해 실제 현장에서 잘 이행될 수 있도록 사명감을 갖고 총력을 기울여 줄 것을 강조하였다.\n 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다.\n 이어, 서울지방철도특별사법경찰대를 방문한 노 장관은 철도시설 내 국민의 지킴이 역할과 해외입국자 KTX 전용칸 수송지원 업무를 차질 없이 수행하고 있는 철도경찰의 노고에 감사드린다면서,\n 코로나19로 인해 철도 승객 간 분쟁이 증가하는 등 철도경찰의 역할이 그 어느 때보다 중요한 때라며, 승객의 방역수칙 위반에 대해서는 무관용 원칙을 적용하여 국민들이 안심하고 철도를 이용할 수 있는 환경을 조성해 줄 것을 당부하였다
'''

max_input_length = 1024
max_target_length = 128

model_inputs = tokenizer(
        text, max_length=max_input_length, truncation=True
    )

predictions = model.generate(torch.tensor([model_inputs['input_ids']]), max_length = max_input_length)

pred = tokenizer.batch_decode(predictions, skip_special_tokens=True)
pred

['노 국토교통부 장관은 철도 분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다고 말했다.']

In [15]:
import torch

!pip install keras-nlp --upgrade

!pip install rouge-score --upgrade

from tensorflow import keras
import rouge_score
import keras_nlp
from keras_nlp.metrics.rouge_base import RougeBase
rouge_l = keras_nlp.metrics.RougeL()
y_true = "노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다. 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다."
y_pred = '노 국토교통부 장관은 철도 분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다고 말했다.'
rouge_l(y_true, y_pred)["f1_score"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 337 kB 19.8 MB/s 
     |████████████████████████████████| 5.8 MB 60.8 MB/s 
     |████████████████████████████████| 588.3 MB 21 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 5.9 MB 61.6 MB/s 
     |████████████████████████████████| 578.1 MB 8.0 kB/s 
     |████████████████████████████████| 578.1 MB 6.8 kB/s 
     |████████████████████████████████| 4.6 MB 64.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=7558c73b8db8e7e44035635e3b102038bbd92bcf10db2b8094826b9ece76b385
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>